In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:
import numpy as np
np.bool = np.bool_

In [4]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [5]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [6]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [7]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executorEnv.PYTHONPATH", "/path/to/your/venv/lib/python3.11/site-packages") \

    .enableHiveSupport()
    .getOrCreate()
)

25/03/12 18:05:27 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/03/12 18:05:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d64e42ad-ffbf-4ff6-b8b6-0dd15bba14d8;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central
	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 146ms :: artifacts dl 14ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 

In [8]:

trump_df = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/trump_btc/trump").cache()


In [9]:
trump_df.toPandas()


,date,favorites,id,isRetweet,retweets,text
0,2024-11-04 04:08:42,0,113422481640840196,True,0,"""RT @TeamTrumpPRESIDENT TRUMP: I’d like to begin by asking a question. Are you better off now than you were 4 years ago?With your vote on Tuesday"
1,2024-11-04 00:32:34,12350,113421631757789522,False,3124,"""KAMALA BROKE IT, BUT I WILL FIX IT! With your VOTE, inflation will END. The border will be SAFE &amp; SECURE. We will have PEACE across the globe. WITH YOUR VOTE, AMERICA WILL ENTER A NEW GOLDEN AGE! Vote TRUMP to MAKE AMERICA GREAT AGAIN! https://swampthevoteusa.com/"""
2,2024-11-03 19:15:28,0,113420384878459920,True,0,"""RT @TeamTrumpPRESIDENT TRUMP: IN CONCLUSION"
3,2024-11-03 06:25:42,17960,113417358038328332,False,5386,"If Kamala wins, you are 3 days away from the start of a 1929-style economic depression. If I win, you are 3 days away from the best jobs, the biggest paychecks, and the brightest economic future the world has ever seen.Kamala’s inflation nightmare has cost the typical American family $30,000 dollars in higher prices—and now, she wants to impose the largest tax hike in American history, and raise your taxes by $3,000 dollars a year. I will massively cut taxes for workers and small businesses—and we will have NO TAX ON TIPS, NO TAX ON OVERTIME, and NO TAX ON SOCIAL SECURITY benefits!"
4,2024-11-02 02:02:14,8165,113410659723667193,False,1919,"Patrick Morrisey, of West Virginia, has been a wonderful Attorney General, and will now be an incredible Governor! He will fight tirelessly to Grow the Economy, Stop Inflation, Secure our Border, Restore American Energy DOMINANCE, Strengthen our Military/Vets, and Protect our always under siege Second Amendment. Patrick will work closely with me to enact our America First Agenda, and is a FANTASTIC person to replace my friend and future U.S. Senator, Jim Justice. Patrick Morrisey is my friend, and has my Complete and Total Endorsement - HE WILL NOT LET YOU DOWN!"
...,...,...,...,...,...,...
846,2011-08-22 22:33:46,193,105739460074938368,False,299,We don't need another stimulus. The first one was a complete failure. Why repeat the same mistake?
847,2011-08-04 22:09:10,24,99210286686408704,False,182,China demanded that we raise our debt ceiling and then their rating agency downgraded us. Our leaders are hope… (cont) http://deck.ly/~x0TDx
848,2011-07-25 21:41:31,14,95579450183385090,False,69,The pressure on the debt ceiling is on @BarackObama.... http://t.co/9gFfiFM #trumpvlog
849,2011-07-14 15:36:02,10,91501208895881216,False,59,"""@johnboehner Debt Ceiling- """"Sometimes no deal is better than a bad deal."""""""


In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    if text:  
        return analyzer.polarity_scores(text)["compound"]
    return 0.0  

trump_tweet_with_sentiment_analysis=trump_df.toPandas()

trump_tweet_with_sentiment_analysis["sentiment_score"] = trump_tweet_with_sentiment_analysis["text"].apply(analyze_sentiment)

trump_tweet_with_sentiment_analysis = spark.createDataFrame(trump_tweet_with_sentiment_analysis)

# Show results
trump_tweet_with_sentiment_analysis.toPandas()




,date,favorites,id,isRetweet,retweets,text,sentiment_score
0,2024-11-04 04:08:42,0,113422481640840196,True,0,"""RT @TeamTrumpPRESIDENT TRUMP: I’d like to begin by asking a question. Are you better off now than you were 4 years ago?With your vote on Tuesday",0.6597
1,2024-11-04 00:32:34,12350,113421631757789522,False,3124,"""KAMALA BROKE IT, BUT I WILL FIX IT! With your VOTE, inflation will END. The border will be SAFE &amp; SECURE. We will have PEACE across the globe. WITH YOUR VOTE, AMERICA WILL ENTER A NEW GOLDEN AGE! Vote TRUMP to MAKE AMERICA GREAT AGAIN! https://swampthevoteusa.com/""",0.9760
2,2024-11-03 19:15:28,0,113420384878459920,True,0,"""RT @TeamTrumpPRESIDENT TRUMP: IN CONCLUSION",0.0000
3,2024-11-03 06:25:42,17960,113417358038328332,False,5386,"If Kamala wins, you are 3 days away from the start of a 1929-style economic depression. If I win, you are 3 days away from the best jobs, the biggest paychecks, and the brightest economic future the world has ever seen.Kamala’s inflation nightmare has cost the typical American family $30,000 dollars in higher prices—and now, she wants to impose the largest tax hike in American history, and raise your taxes by $3,000 dollars a year. I will massively cut taxes for workers and small businesses—and we will have NO TAX ON TIPS, NO TAX ON OVERTIME, and NO TAX ON SOCIAL SECURITY benefits!",0.7861
4,2024-11-02 02:02:14,8165,113410659723667193,False,1919,"Patrick Morrisey, of West Virginia, has been a wonderful Attorney General, and will now be an incredible Governor! He will fight tirelessly to Grow the Economy, Stop Inflation, Secure our Border, Restore American Energy DOMINANCE, Strengthen our Military/Vets, and Protect our always under siege Second Amendment. Patrick will work closely with me to enact our America First Agenda, and is a FANTASTIC person to replace my friend and future U.S. Senator, Jim Justice. Patrick Morrisey is my friend, and has my Complete and Total Endorsement - HE WILL NOT LET YOU DOWN!",0.9824
...,...,...,...,...,...,...,...
846,2011-08-22 22:33:46,193,105739460074938368,False,299,We don't need another stimulus. The first one was a complete failure. Why repeat the same mistake?,-0.6908
847,2011-08-04 22:09:10,24,99210286686408704,False,182,China demanded that we raise our debt ceiling and then their rating agency downgraded us. Our leaders are hope… (cont) http://deck.ly/~x0TDx,-0.5267
848,2011-07-25 21:41:31,14,95579450183385090,False,69,The pressure on the debt ceiling is on @BarackObama.... http://t.co/9gFfiFM #trumpvlog,-0.5719
849,2011-07-14 15:36:02,10,91501208895881216,False,59,"""@johnboehner Debt Ceiling- """"Sometimes no deal is better than a bad deal.""""""",-0.6486


In [11]:

(trump_tweet_with_sentiment_analysis.write 
                    .format("delta")
                    .mode("overwrite")
                    .option("path",f"{hdfs_lakehouse_base_path}/gold/trump_btc/trump_tweet_with_sentiment_analysis")
                    .saveAsTable("trump_btc.trump_tweet_with_sentiment_analysis")
)
